# Gradio

How to serve the model in less than 10 lines of code

In [2]:
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

Having a great model is only one of the many steps toward a successfull application of deep-learning to a problem  
In this notebook we will see how to serve it to the world and allow others to use it

Let's try to serve the previously trained MNIST model using *LeCun* CNN architecture

In [3]:
class LeCunRevisited(nn.Module):

    def __init__(self):
        super(LeCunRevisited, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.max_pool = nn.MaxPool2d(kernel_size=2)

    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    
model = LeCunRevisited()

We load the weight of the saved neural network

In [4]:
model.load_state_dict(torch.load('mnist.pth'))

<All keys matched successfully>

We then use **Gradio** to create a web interface in less than 10 lines of code

In [8]:
import gradio as gr

# we need to give a name to our labels
labels = [str(i) for i in range(10)]

# define the function that will be called when an image is send to our application
def predict(img):
    # Gray scale because MNIST are black and white images
    img = torchvision.transforms.Grayscale(num_output_channels=1)(torchvision.transforms.ToTensor()(img).unsqueeze(0))
    with torch.inference_mode():
        prediction = torch.nn.functional.softmax(model(img)[0], dim=0)
        confidences = {labels[i]: float(prediction[i]) for i in range(len(labels))}
    return confidences

In [10]:
title = "Handwritten Digits Classifier"
description = "An handwritten digits classifier using LeCun CNN architecture."
iface = gr.Interface(predict, 
                     inputs=gr.Image(shape=(28, 28)), 
                     outputs=gr.Label(num_top_classes=3), 
                     live=True, 
                     title=title,
                     description=description,
                     interpretation='default').launch(share=True)

Hint: Set streaming=True for Image component to use live streaming.
Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://9e218ec3cba0af34.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/Users/ingambe/miniforge3/envs/deep-learning/lib/python3.9/site-packages/gradio/routes.py", line 284, in run_predict
    output = await app.blocks.process_api(
  File "/Users/ingambe/miniforge3/envs/deep-learning/lib/python3.9/site-packages/gradio/blocks.py", line 982, in process_api
    result = await self.call_function(fn_index, inputs, iterator)
  File "/Users/ingambe/miniforge3/envs/deep-learning/lib/python3.9/site-packages/gradio/blocks.py", line 824, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/ingambe/miniforge3/envs/deep-learning/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/ingambe/miniforge3/envs/deep-learning/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/Users/ingambe/miniforge3/envs/deep-learning/lib/python3.9/site-pa